In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [7]:
df = pd.read_csv('train.csv')

if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


#Look for the missing values in all the columns and either impute them (replace with mean,median, or mode) or drop them.

New_price was kept because it provides a cleaned numerical representation of car prices which is useful for analysis. It enables accurate comparisons, feature engineering, and modeling. Removing it would lead to loss of valuable pricing insights.

Other missing values were likely retained to allow further analysis, imputation, or transformations based on their importance. Dropping them immediately could result in losing useful data. They might be handled later through filling, encoding, or conditional removal.

In [8]:
missing = df.isnull().sum()
print("Missing values before imputation:\n", missing)

Missing values before imputation:
 Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [9]:
for col in df.columns:
    if col != 'New_Price' and df[col].isnull().mean() > 0.3:
        df = df.drop(columns=col)
    elif df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].median())


#Checking the missing values
print("Missing values after imputation:\n", df.isnull().sum())

Missing values after imputation:
 Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


#  Remove the units from some of the attributes and only keep the numerical values (forexample remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from“New_price”).

In [10]:
def extract_number(value):
    if isinstance(value, str):
        match = re.search(r"[\d.]+", value.replace(',', ''))
        return float(match.group()) if match else None
    return value

# Applying to the relevant columns
for col in ['Mileage', 'Engine', 'Power', 'New_Price']:
    if col in df.columns:
        df[col] = df[col].apply(extract_number)

#df[['Mileage', 'Engine', 'Power', 'New_Price']].head()
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,4.78,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,8.61,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,4.78,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,4.78,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,4.78,3.50


# Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.

In Fuel_type Changed Diesel to 0 and Petrol to 1

In Transmission changed Manual to 1 and Automatic to 0

In [11]:
df['Fuel_Type'] = df['Fuel_Type'].astype(str).str.strip().str.lower()
df['Fuel_Type'] = df['Fuel_Type'].apply(lambda x: 1 if x == 'petrol' else (0 if x == 'diesel' else None))
df['Fuel_Type'] = df['Fuel_Type'].astype('Int64')

# Clean and encode Transmission
df['Transmission'] = df['Transmission'].astype(str).str.strip().str.lower()
df['Transmission'] = df['Transmission'].apply(lambda x: 1 if x == 'manual' else (0 if x == 'automatic' else None))
df['Transmission'] = df['Transmission'].astype('Int64')

df.head()


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,1,First,19.67,1582.0,126.20,5.0,4.78,12.50
1,Honda Jazz V,Chennai,2011,46000,1,1,First,13.00,1199.0,88.70,5.0,8.61,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,0,1,First,20.77,1248.0,88.76,7.0,4.78,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,0,Second,15.20,1968.0,140.80,5.0,4.78,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,0,1,First,23.08,1461.0,63.10,5.0,4.78,3.50


# Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by subtracting “Year” value from the current year.

In [12]:
current_year = 2025
df['Car_Age'] = current_year - df['Year']
#df[['Year', 'Car_Age']].head()
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,1,First,19.67,1582.0,126.20,5.0,4.78,12.50,10
1,Honda Jazz V,Chennai,2011,46000,1,1,First,13.00,1199.0,88.70,5.0,8.61,4.50,14
2,Maruti Ertiga VDI,Chennai,2012,87000,0,1,First,20.77,1248.0,88.76,7.0,4.78,6.00,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,0,Second,15.20,1968.0,140.80,5.0,4.78,17.74,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,0,1,First,23.08,1461.0,63.10,5.0,4.78,3.50,12


# Perform select, filter, rename, mutate, arrange and summarize with group by operations (or their equivalent operations in python) on this dataset.

In [13]:
selected_df = df[['Name', 'Location', 'Year', 'Kilometers_Driven', 'Car_Age', 'Price']]


# Filter: Cars driven less than 50,000 km and newer than 5 years old
filtered_df = df[(df['Mileage'] > 15) & (df['Price'] < 10)]

# Rename: Rename 'Price' to 'Selling_Price' and 'Kilometers_Driven' to 'KMs'
renamed_df = df.rename(columns={'Price': 'Selling_Price', 'Kilometers_Driven': 'KMs'})

#Mutate: Add a new column - Price per Kilometer
df['Price_per_KM'] = df['Price'] / df['Kilometers_Driven']


# Arrange: Sort the dataset by 'Price' descending
arranged_df = df.sort_values(by='Price', ascending=False)


# Summarize: Group by 'Location' and show average and max price
summary = df.groupby('Location').agg(
    Avg_Price=('Price', 'mean'),
    Max_Price=('Price', 'max'),
    Count=('Price', 'count')
).reset_index()


print(" Filtered DataFrame (Less than 50,000 km and < 5 years old):")
display(filtered_df.head())
print("\n\n")

print(" Renamed Columns:")
display(renamed_df.head())
print("\n\n")

print("Mutated Column (Price_per_KM):")
display(df[['Name', 'Kilometers_Driven', 'Price', 'Price_per_KM']].head())
print("\n\n")

print("Arranged by Highest Price:")
display(arranged_df[['Name', 'Price']].head())
print("\n\n")

print("Summary by Location:")
display(summary.head())
print("\n\n")

 Filtered DataFrame (Less than 50,000 km and < 5 years old):


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age
2,Maruti Ertiga VDI,Chennai,2012,87000,0,1,First,20.77,1248.0,88.76,7.0,4.78,6.00,13
4,Nissan Micra Diesel XV,Jaipur,2013,86999,0,1,First,23.08,1461.0,63.10,5.0,4.78,3.50,12
6,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,0,1,First,20.54,1598.0,103.60,5.0,4.78,5.20,12
7,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,0,1,Second,22.30,1248.0,74.00,5.0,4.78,1.95,13
8,Maruti Ciaz Zeta,Kochi,2018,25692,1,1,First,21.56,1462.0,103.25,5.0,10.65,9.95,7





 Renamed Columns:


,Name,Location,Year,KMs,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Selling_Price,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,0,1,First,19.67,1582.0,126.20,5.0,4.78,12.50,10
1,Honda Jazz V,Chennai,2011,46000,1,1,First,13.00,1199.0,88.70,5.0,8.61,4.50,14
2,Maruti Ertiga VDI,Chennai,2012,87000,0,1,First,20.77,1248.0,88.76,7.0,4.78,6.00,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,0,0,Second,15.20,1968.0,140.80,5.0,4.78,17.74,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999,0,1,First,23.08,1461.0,63.10,5.0,4.78,3.50,12





Mutated Column (Price_per_KM):


,Name,Kilometers_Driven,Price,Price_per_KM
0,Hyundai Creta 1.6 CRDi SX Option,41000,12.50,0.000305
1,Honda Jazz V,46000,4.50,0.000098
2,Maruti Ertiga VDI,87000,6.00,0.000069
3,Audi A4 New 2.0 TDI Multitronic,40670,17.74,0.000436
4,Nissan Micra Diesel XV,86999,3.50,0.000040





Arranged by Highest Price:


,Name,Price
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,160.00
5620,Lamborghini Gallardo Coupe,120.00
5752,Jaguar F Type 5.0 V8 S,100.00
1457,Land Rover Range Rover Sport SE,97.07
1917,BMW 7 Series 740Li,93.67





Summary by Location:


,Location,Avg_Price,Max_Price,Count
0,Ahmedabad,8.567248,85.00,218
1,Bangalore,13.482670,93.00,352
2,Chennai,7.958340,70.00,476
3,Coimbatore,15.160206,93.67,631
4,Delhi,9.881944,120.00,540
